<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/classifier101_scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai h5py pillow
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4
import numpy as np
import pandas as pd
import os
from fastai.vision.all import *

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/kaggle/isic-2024-data/


cnn.pkl			 test-image.hdf5       train-image.hdf5
random_forest_model.pkl  test-metadata.csv     train-metadata.csv
sample_submission.csv	 test-metadata.gsheet  train-metadata-v1.csv


In [ ]:
HDF5_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-image.hdf5"
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"

In [ ]:
import pandas as pd

# Load the metadata CSV file
metadata_df = pd.read_csv(METADATA_FILE)
# Display the first few rows to understand the structure

metadata_df = metadata_df[:100000]
# metadata_df[metadata_df['target'] == 1]

metadata_df

<ipython-input-5-2b8c2615d435>:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,ISIC_2550103,0,IP_9025934,45.0,male,lower extremity,2.50,TBP tile: close-up,3D: XP,22.788070,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999990
99996,ISIC_2550129,0,IP_7370330,25.0,female,head/neck,3.51,TBP tile: close-up,3D: XP,24.341550,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
99997,ISIC_2550131,0,IP_5294508,55.0,male,lower extremity,2.60,TBP tile: close-up,3D: white,25.519840,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,98.708180
99998,ISIC_2550144,0,IP_8935276,65.0,male,posterior torso,9.20,TBP tile: close-up,3D: white,14.385220,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,98.477860


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming metadata_df has columns 'isic_id' and 'target'
train_df, test_df = train_test_split(metadata_df, test_size=0.1, random_state=42)  # 20% for testing

target_size = (56, 56)
batch_size = 64
input_shape=(56, 56, 3)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setup ImageDataGenerator with augmentations
datagen = ImageDataGenerator(
    rotation_range=60,            # Random rotations up to 30 degrees
    width_shift_range=0.2,        # Horizontal shifts up to 10% of the width
    height_shift_range=0.2,       # Vertical shifts up to 10% of the height
    brightness_range=[0.8, 1.2],  # Random brightness adjustment
    zoom_range=0.5,               # Random zooms up to 20%
    horizontal_flip=True,         # Random horizontal flips
    fill_mode='nearest'           # Fill mode for points outside the boundaries
)


In [ ]:
import h5py
import numpy as np
import cv2

def load_and_preprocess_images(image_bytes, target_size):

  # Decode the bytes into an image array
  image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)

  # Resize the image to the target size
  resized_image = cv2.resize(image, target_size)

  # Normalize the image
  normalized_image = resized_image / 255.0

  # Convert list to NumPy array
  features = np.array(normalized_image)

  return features


In [ ]:

import h5py
import numpy as np
import cv2

def hdf5_data_generator(hdf5_file, metadata_df, batch_size, target_size=(56, 56)):
  with h5py.File(hdf5_file, 'r') as hf:
    num_samples = len(train_df)

    while True:
      for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)

        batch_isic_ids = train_df['isic_id'].iloc[start:end]
        batch_labels = train_df['target'].iloc[start:end].values

        batch_images = []

        for isic_id in batch_isic_ids:
          image_bytes = hf[str(isic_id)][()]
          features = load_and_preprocess_images(image_bytes, target_size)
          batch_images.append(features)

        # Apply augmentations using datagen
        augmented_images = np.array([datagen.random_transform(image) for image in batch_images])

        yield augmented_images, batch_labels

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight

# Load the pre-trained EfficientNetB0 model without the top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Single output for binary classification
])

# Calculate class weights based on the training set
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['target']), y=train_df['target'])
class_weight_dict = dict(enumerate(class_weights))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(hdf5_data_generator(HDF5_FILE, train_df, batch_size),
          steps_per_epoch=len(train_df) // 32,
          epochs=2)  # Start with fewer epochs to see how it performs

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/2
2812/2812 ━━━━━━━━━━━━━━━━━━━━ 1186s 417ms/step - accuracy: 0.9973 - loss: 0.0115
Epoch 2/2
2812/2812 ━━━━━━━━━━━━━━━━━━━━ 1162s 413ms/step - accuracy: 0.9991 - loss: 0.0082


In [11]:
import h5py
import numpy as np
import cv2
from tensorflow.keras.models import load_model


results = []

with h5py.File(HDF5_FILE, 'r') as hf:
    num_samples = len(test_df)

    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        batch_df = test_df.iloc[start:end]
        batch_isic_ids = batch_df['isic_id'].values
        batch_images = []

        for isic_id in batch_isic_ids:
          image_bytes = hf[str(isic_id)][()]

          # Decode the bytes into an image array
          image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)

          # Resize the image to the target size
          resized_image = cv2.resize(image, target_size)

          # Normalize the image
          normalized_image = resized_image / 255.0

          # Add a batch dimension to the image (from (56, 56, 3) to (1, 56, 56, 3))
          image_batch = np.expand_dims(normalized_image, axis=0)

          # Make a prediction
          prediction = model.predict(image_batch) [0]
          results.append({'isic_id': isic_id, 'target': prediction})


results_df = pd.DataFrame(results)

# results_df.to_csv('predictions.csv', index=False)




Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

In [12]:
results_df[results_df['isic_id']]

,isic_id,target
0,ISIC_1946235,[9.541365e-05]
1,ISIC_2056320,[9.574659e-05]
2,ISIC_0561396,[9.563362e-05]
3,ISIC_1970966,[9.5728064e-05]
4,ISIC_2376895,[9.563089e-05]
...,...,...
9995,ISIC_0194565,[9.577225e-05]
9996,ISIC_0814999,[9.569456e-05]
9997,ISIC_2382197,[9.580888e-05]
9998,ISIC_1884291,[9.583666e-05]


In [13]:
# import joblib

# # Save the trained model to a file
# joblib_filename = 'random_forest_model.joblib'
# joblib.dump(model, joblib_filename)


In [14]:
# import pickle

# # Save the trained model to a file
# pickle_filename = '/content/drive/MyDrive/kaggle/isic-2024-data/cnn.pkl'
# with open(pickle_filename, 'wb') as file:
#     pickle.dump(model, file)


In [15]:
# import pickle

# # Load the model from the pkl file
# with open(pickle_filename, 'rb') as file:
#     model = pickle.load(file)


In [16]:
import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC

    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC)
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)

    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)
    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)

    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)

    partial_auc = auc(fpr, tpr)

    return(partial_auc)

# solution = test_df[['isic_id', 'target']]
# predictions = predictions_df[['isic_id', 'target']]

# partial_auc = score(solution, predictions, "isic_id")
# partial_auc